https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [3]:
from langchain.schema import Document
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
import feedparser

KEYWORD = "RAG"
# ArXiv API URL
url = f'http://export.arxiv.org/api/query?search_query=abs:{KEYWORD}&start=0&max_results=10&sortBy=lastUpdatedDate&sortOrder=descending'
# Parse the API response
feed = feedparser.parse(url)
data = feed.entries

docs = []
for paper in data:
    title = paper["title"]
    abstract = paper["summary"]
    link = paper["link"]
    paper_content = f"Title: {title}\nAbstract: {abstract}"
    paper_content = paper_content.lower()

    docs.append(Document(page_content=paper_content,
                            metadata={"link": link}))
    
embedding_model = CohereEmbeddings(model="embed-english-light-v3.0")

vector_db = Chroma.from_documents(
    documents=docs, embedding=embedding_model)

vector_db.add_documents(docs)

retriever = vector_db.as_retriever()

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {context}"),
    ("human", "{input}"),
])

In [5]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You're an assistant who's good at {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [6]:
from langchain_cohere import ChatCohere
llm = ChatCohere(model="command-r-plus-08-2024",
           max_tokens=256,
           temperature=0.5
           )

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser

# NOTE: default input vars must be 'context' and 'input' for the prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [8]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

TypeError: __init__() takes 1 positional argument but 2 were given